In [58]:
import numpy as np
import pandas as pd
import plotly.express as px

dane = pd.read_csv('cleaned.csv')
print(dane.head(10))

       Brand                                Product_Description  Screen_Size  \
0     Lenovo  Lenovo ThinkPad 14” HD Laptop PC Computer Core...         14.0   
1       Dell  Dell Latitude 15.6" Laptop Intel Core i5 64GB ...         15.6   
2         HP  NEW HP 15 Laptop. 15.6" 1080p, i5-1135G7, 8GB ...         15.6   
3     Lenovo  2024 Lenovo Ideapad Laptop 15 FHD Touch Intel ...         15.6   
4     Lenovo  2024 Lenovo Ideapad Laptop 15.6" FHD Touch Int...         15.6   
5  Microsoft  Microsoft Surface Laptop 5 13.5" - 512GB SSD, ...         13.5   
6     Lenovo  Lenovo V15 G4 IRU 15.6" Full HD Laptop, i7-135...         15.6   
7       Dell  Dell Latitude 15.6" Laptop Intel Core i5 64GB ...         15.6   
8  Microsoft  Microsoft Surface Laptop 2 -8th Gen - i5 - 1.7...         13.5   
9     Lenovo  Lenovo ThinkPad T470 14' LCD i5 Laptop PC 16GB...         14.0   

  RAM                Processor                     GPU  \
0  16   Intel Core i5 7th Gen.   Intel HD Graphics 520   
1  

In [21]:
dane.shape, dane.dtypes

((2952, 10),
 Brand                   object
 Product_Description     object
 Screen_Size            float64
 RAM                     object
 Processor               object
 GPU                     object
 GPU_Type                object
 Resolution              object
 Condition               object
 Price                  float64
 dtype: object)

# Opis zbioru danych 
Nasz dataset ma 2952 wiersze i 10 kolumn. Jest to zbiór dotyczący laptopów - ich cen, specyfikacji sprzętowej oraz opisu. Każda z kolumn opisuje zmienną w sposób następujący:
1. **Brand** - zmienna kategoryczna opisująca markę laptopa
2. **Product_Description** - zmienna kategoryczna; są to opisy poszczególnych modeli
3. **Screen_Size** - zmienna ilościowa; rozmiar ekranu
4. **RAM** - zmienna kategoryczna; pamięć RAM laptopa
5. **Processor** - zmienna kategoryczna; model procesora
6. **GPU** - zmienna kategoryczna; model karty graficznej
7. **GPU_Type** - zmienna kategoryczna; typ karty graficznej
8. **Resolution** - zmienna kategoryczna; rozdzielczość ekranu 
9. **Condition** - zmienna kategoryczna; kondycja sprzętu
10. **Price** - zmienna ilościowa; cena laptopa

# Analiza zmiennych w zbiorze danych

In [19]:
dane.describe()

,Screen_Size,Price
count,2952.000000,2952.000000
mean,14.550237,517.552805
std,1.175322,235.592872
min,10.000000,0.000000
25%,14.000000,325.990000
50%,14.000000,489.990000
75%,15.600000,699.000000
max,18.000000,999.990000


Z powyższych statystyk możemy już wyciągnąć parę ciekawych wniosków. Po pierwsze, jest stosunkowo niewielki rozstrzał rozmiaru ekranu, bo nieco ponad 1 cal w jedną i drugą stronę. Zatem możemy powiedzieć, że jest już na rynku pewien "standardowy" rozmiar ekranu, a znacznie większe ekrany w laptopach są rzadkością. Interesujące może być również to, że istnieje laptop, którego cena wynosi... okrągłe zero. Zastanawia również fakt, że tak często pojawiają się ceny z 9 na końcu, co jest popularnym, acz przestarzałym w mojej opinii chwytem marketingowym. 

## Teraz zbadamy rozkład ilości poszczególnych marek w zbiorze

In [68]:
dane.Brand.unique().shape
brands = dane.groupby("Brand").Product_Description.agg("count")
brands = pd.DataFrame(brands)
brands.Product_Description.sum() == 2952
brands.head(10)


,Product_Description
Brand,
AVITA,1
Acer,100
Apple,5
Asus,134
ByteSpeed,1
Chuwi,2
Dell,1109
Dell Inc.,2
Eurocom,1


Zatem widzimy, że algorytm dobrze policzył i suma wszystkich wartości w wierszach wynosi 2952, czyli tyle ile jest w wierszy w datasecie. Zobaczmy rozkład dla zmiennej Brand.

In [67]:
fig = px.bar(brands,
            x = "Product_Description",
            title = "Distribution of brands",
            labels = {
                "Product_Description": "Count"
            })
fig.show()

Widzimy z powyższego wykresu, że laptopy marki Dell dominują ramkę danych. Zaraz za nimi są urządzenia firmy Lenovo i HP. Na szarym końcu są takie marki jak: Apple, Getac, Google, Sony, Ruggon, Panasonic, Huawei, Intel, Gigabyte, Gateway.

# Analiza zmiennych w zależności od przynależności do grupy

In [85]:
dane.groupby(["Processor", "GPU"]).Price.agg("mean").sort_values(ascending = False).head(15)

Processor                GPU                          
Intel Core i7 11th Gen.  NVIDIA GeForce RTX 2060 Max-Q    999.99
Intel Core i7 6th Gen.   nvidia geforce gtx 970m          999.99
Intel Core i7 11th Gen.  Nvidia GTX 1650 TI 4GB           999.99
Intel Core i9 12th Gen.  NVIDIA GeForce RTX 3070          999.99
Intel Core i7 13th Gen.  NVIDIA GeForce RTX A1000         999.95
Intel Core i7 12th Gen.  NVIDIA GeForce                   999.88
Intel Core i7 10th Gen.  NVIDIA Quadro RTX 5000           999.00
                         NVIDIA Quadro T2000              999.00
Intel Core i7 11th Gen.  NVIDIA® T1200 4GB                989.99
                         NVIDIA RTX A4000                 980.00
Intel Core i7 12th Gen.  NVIDIA GeForce MX450             975.00
Intel Core i7 13th Gen.  nVidia GeForce RTX 4060          974.95
Intel Core i9 11th Gen.  NVIDIA GeForce RTX 3050 Ti       960.36
Intel Core i7 13th Gen.  Integrated Graphics              955.00
Intel Core i7 10th Gen.  Intel Iris

Zauważmy, że najdroższe laptopy mają najlepsze i najnowsze procesory Intel Core i7 lub i9 najnowszej generacji (6 w zwyż). Ponadto dobry procesor musi iść w parze z bardzo dobrą kartą graficzną, choć nie musi to być reguła, ponieważ nvidia geforce gtx 970m nie jest uważana za dobrą kartę graficzną. Zasadniczo jednak dobry procesor idzie w parze z dobrą kartą graficzną.

## Teza: większość laptopów ma zintegrowaną kartę graficzną
Sprawdzimy prawdziwość tej tezy

In [80]:
dane.GPU_Type.unique()

array(['Integrated/On-Board Graphics', 'Dedicated Graphics', nan,
       'Intel UHD Graphics', 'Intergrated Intel UHD Graphics',
       'Intel Iris Xe Graphics', 'Iris Xe Graphics',
       'Intel® UHD Graphics', 'Integrated'], dtype=object)

Widzimy, że w kolumnie z typem karty graficznej występują 3 rodzaje zintegrowanej karty, więc na końcu zsumujemy je

In [83]:
dane.groupby("GPU_Type").Price.agg("count").head(10)

GPU_Type
Dedicated Graphics                 445
Integrated                           2
Integrated/On-Board Graphics      2265
Intel Iris Xe Graphics               1
Intel UHD Graphics                   1
Intel® UHD Graphics                  1
Intergrated Intel UHD Graphics       1
Iris Xe Graphics                     2
Name: Price, dtype: int64

Stąd, po prostym rachunku dodawnia liczb całkowitych w pamięci, dostajemy, że:
1. **Zintegrowane karty**: 2268
2. **Dedykowane**: 684
3. **Łącznie**: 2952 (czyli tyle ile wierszy, więc się zgadza)
werdykt: teza prawdziwa

In [90]:
dane.groupby("RAM").Price.agg("mean").sort_values(ascending = False).head(10)

RAM
16GB,    899.990000
256      769.890000
512      650.000000
64gb     649.990000
upto     649.000000
8GB,     632.623333
8GB      594.481429
16       574.896851
32       569.662967
up       565.666667
Name: Price, dtype: float64

Powszechnie uważa się, że wraz z większą ilością RAM-u rośnie cena laptopa. Czy tak jest w rzeczywistości? 
Z powyższej analizy niewiele wynika, bo mamy dwie wartości dla 16gb ramu, co może fałszować ostateczny werdykt. Usunę GB na końcu liczby.


In [92]:
dane["RAM"] = dane["RAM"].str.replace('GB,', '').str.replace('gb', '').str.replace('GB', '')
dane.groupby("RAM").Price.agg("mean").sort_values(ascending = False).head(10)

RAM
256     769.890000
512     650.000000
upto    649.000000
16      573.841726
32      569.043723
up      565.666667
24      544.495000
64      526.463542
20      478.325000
40      464.995000
Name: Price, dtype: float64

Zatem po zmianach, jakie wprowadziliśmy otrzymujemy, że owszem, pamięć RAM idzie w parze ze średnią ceną, natomiast tylko dla wyższych wartości RAM. Między np. 16, a 32 nie ma dużej różnicy.

# Wykresy analizujące rozkład zmiennych w zbiorze 

In [98]:
fig = px.histogram(dane,
                  x = "Price",
                    title = "Distribution of Prices",
                  labels ={
                      "count": "Count"
                  },
                  hover_name = "Price")
fig.show()

Nie widać wyraźnie jakiegoś konkretnego rozkładu. Wydawać by się mogło nawet, że zmienne pomiędzy 200 a 1000 w paru miejscach "wybuchają" i jest ich w pewnych kubełkach nagle więcej pod pozostałych do okoła. Możemy za to powiedzieć, że najmniej jest laptopów o cenie między 0-200 dolarów. Najwięcej między 480-499.99 (znów chwyt marketingowy). Co więcej, w ostatnim kubełku wartości również jest niemało, z uwagi na fakt, że sprzedawcy wolą zostawić cenę 999.99 niż podnosić do 1000.

In [105]:
fig = px.box(dane,
                x = "Screen_Size",
                labels ={
                      "count": "Count"
                  },
                title = "Boxplot of Screen Sizes",
            orientation = "h")
fig.show()

Krótko, zwięźle i na temat: outliery, to 10, 10.1, 10.5 (naprawdę małe ekrany), mediana równa 14, wartość największa równa 18 (naprawdę duży ekran - 18 cali to około 45 cm).

# Wykresy wskazujące zależności w zbiorze danych z wnioskami

## Czy istnieje korelacja pomiędzy rozmiarem ekranu, a ceną?

In [109]:
correlation_df = dane.loc[:, ["Screen_Size", "Price"]].corr().round(3)
fig = px.imshow(correlation_df, 
          color_continuous_scale = px.colors.sequential.RdBu_r, 
          title = "Correlation of Screen Size and Price",
          zmin = -1,
          zmax = 1,
          text_auto = True)
fig.show()

Zatem zależność jest bardzo niewielka, prawie niezauważalna

In [134]:
dane_2 = dane.loc[dane['Brand'].isin(["Dell", "Lenovo", "HP"])]
dane_2_grouped = pd.DataFrame(dane_2.groupby(["Brand", "Condition"]).Price.agg("mean").reset_index())

fig = px.bar(dane_2_grouped,
            x = "Brand",
            y = "Price",
            color = "Condition",
            barmode = "group",
            title = "Barplot of Price, Condition and Brand - comparison")
fig.show()

Na ostatnim wykresie mamy porównanie 3 najpopularniejszych marek w datasecie: Dell, HP i Lenovo. Wykres obrazuje jak zmienia się cena w zależności od marki oraz stanu produktu.
1. Dla nowych urządzeń (zielony słupek) cena jest porównywalna dla wszystkich 3 marek
2. Między czerwonymi, pomarańczowymi słupkami też nie ma dużej różnicy
3. Jedyna dysproporcja na wykresie jest pomiędzy excellent dla Dell i HP oraz open box dla Dell i Lenovo - wniosek: sprzęty marki HP o stanie excellent są o około 100 dolarów droższe oraz laptopy lenovo o stanie open box są również o około 100 dolarów tańsze od Dell.